In [1]:
import os
from datetime import datetime
import time

import pandas as pd
import numpy as np

import plotly.figure_factory as ff
import plotly.express as px

In [2]:
datafile: pd.DataFrame = pd.read_csv("data/president_state.csv")
state_codes: pd.DataFrame = pd.read_csv("data/world_country_and_usa_states_latitude_and_longitude_values.csv")


In [3]:
datafile = datafile.merge(right=state_codes, left_on="state", right_on="usa_state")

In [4]:
fig = px.choropleth(datafile, locations='usa_state_code', color="total_votes",
                           range_color=(0, 10000000),
                           locationmode = 'USA-states',  
                           scope="usa",
                           title='USA Presidential Votes Counts' 
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [5]:
df_president_county = pd.read_csv("data/president_county_candidate.csv")

In [6]:
df_president_county

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True
...,...,...,...,...,...,...
32172,Arizona,Maricopa County,Write-ins,WRI,1331,False
32173,Arizona,Mohave County,Donald Trump,REP,78535,True
32174,Arizona,Mohave County,Joe Biden,DEM,24831,False
32175,Arizona,Mohave County,Jo Jorgensen,LIB,1302,False


In [7]:
df_president_county.groupby(["state", "county"])["total_votes"].aggregate(np.sum)

state    county           
Alabama  Autauga County        27770
         Baldwin County       109679
         Barbour County        10518
         Bibb County            9595
         Blount County         27588
                               ...  
Wyoming  Sweetwater County     16603
         Teton County          14677
         Uinta County           9402
         Washakie County        4012
         Weston County          3542
Name: total_votes, Length: 4633, dtype: int64

In [8]:
df_president_county.groupby(["state", "county", "candidate"])["total_votes"].aggregate(np.sum) / df_president_county.groupby(["state", "county"])["total_votes"].aggregate(np.sum)

state    county          candidate   
Alabama  Autauga County   Write-ins      0.002845
                         Donald Trump    0.714368
                         Jo Jorgensen    0.012604
                         Joe Biden       0.270184
         Baldwin County   Write-ins      0.002991
                                           ...   
Wyoming  Weston County    Write-ins      0.002541
                         Brock Pierce    0.005647
                         Donald Trump    0.877188
                         Jo Jorgensen    0.012987
                         Joe Biden       0.101637
Name: total_votes, Length: 32177, dtype: float64

In [9]:
df = df_president_county.groupby(["state", "county", "party"])["total_votes"].aggregate(np.sum) / df_president_county.groupby(["state", "county"])["total_votes"].aggregate(np.sum)

In [10]:
df = df.to_frame()
df = df.reset_index(level=["state", "county", "party"])
df

,state,county,party,total_votes
0,Alabama,Autauga County,DEM,0.270184
1,Alabama,Autauga County,LIB,0.012604
2,Alabama,Autauga County,REP,0.714368
3,Alabama,Autauga County,WRI,0.002845
4,Alabama,Baldwin County,DEM,0.224090
...,...,...,...,...
29975,Wyoming,Weston County,DEM,0.101637
29976,Wyoming,Weston County,IND,0.005647
29977,Wyoming,Weston County,LIB,0.012987
29978,Wyoming,Weston County,REP,0.877188


In [11]:
df_party_max_votes = df.groupby(["state", "county"])["total_votes"].max().reset_index()

In [12]:
df = df.merge(df_party_max_votes, on=["state", "county", "total_votes"])

In [13]:
df

,state,county,party,total_votes
0,Alabama,Autauga County,REP,0.714368
1,Alabama,Baldwin County,REP,0.761714
2,Alabama,Barbour County,REP,0.534512
3,Alabama,Bibb County,REP,0.784263
4,Alabama,Blount County,REP,0.895716
...,...,...,...,...
4642,Wyoming,Sweetwater County,REP,0.736554
4643,Wyoming,Teton County,DEM,0.670982
4644,Wyoming,Uinta County,REP,0.797277
4645,Wyoming,Washakie County,REP,0.808824
